## <6-2 개체명 인식 모델 학습하기>  

개체명 인식 데이터 전처리와 모델의 학습 과정을 살펴보겠습니다.

### 개체명을 인식하는 모델 만들기  
---
<**1단계**> **코랩 노트북 초기화하기**   
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3oMA8L1)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에 접속한 후, `[내 드라이브에 복사]`와 런타임 유형에서 하드웨어 가속을 `[GPU]`로 선택합니다. 

<**2단계**> **각종 설정하기**  
1단계 코랩 노트북 초기화 과정에서 하드웨어 가속기를 TPU로 선택했다면 다음 코드를 실행하고, GPU를 선택했다면 실행하지 않습니다.

In [1]:
#코드 6-1 TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

다음 코드를 실행해 TPU 이외에 의존성 있는 패키지를 설치하고 자신의 구글 드라이브를 코랩 노트북과 연결합니다.

In [2]:
#코드 6-2 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 906 kB/s 
     |████████████████████████████████| 2.8 MB 18.6 MB/s 
     |████████████████████████████████| 806 kB 60.1 MB/s 
     |████████████████████████████████| 57 kB 7.2 MB/s 
     |████████████████████████████████| 136 kB 74.2 MB/s 
     |████████████████████████████████| 829 kB 61.8 MB/s 
     |████████████████████████████████| 398 kB 67.1 MB/s 
     |████████████████████████████████| 636 kB 64.2 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 895 kB 53.2 MB/s 
     |████████████████████████████████| 3.3 MB 7.8 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 96 kB 8.1 MB/s 
     |████████████████████████████████| 144 kB 75.3 MB/s 
     |████████████████████████████████| 271 kB 71.2 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [3]:
#코드 6-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 이준범 님이 공개한 `kcbert-base` 모델을 실습 데이터로 파인튜닝해 볼 예정입니다.  
실습 데이터로는 한국해양대학교 자연언어처리연구실에서 공개한 데이터와 자체 제작한 데이터(`ner`)를 합쳐 사용합니다.  
다음 코드를 실행하면 관련 설정을 할 수 있습니다. 개체명 인식을 위한 환경 설정은 문서 분류와 본질적으로 다르지 않습니다. 이 코드에서 설정하는 각종 인자에 관한 설명은 <4-2>절을 참고 바랍니다.

In [4]:
#코드 6-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

이어서 다음 코드를 실행해 랜덤 시드를 고정하고 로거를 설정합니다.

In [5]:
#코드 6-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [6]:
#코드 6-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


<**3단계**> **말뭉치 내려받기**  
다음 코드를 실행하면 `corpus_name(ner)`애 해당하는 말뭉치를 내려받습니다.

In [7]:
#코드 6-7 말뭉치 다운로드
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 47.7MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 43.4MB/s]


<**4단계**> **토크나이저 준비하기**  
다음 코드를 실행해 `kcbert-base` 모델이 사용하는 토크나이저를 선언합니다.

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

<**5단계**> **데이터 전처리하기**  
다음 코드를 수행하면 데이터셋을 만들 수 있습니다. `NERCorpus` 클래스는 텍스트 파일을 '원본 문장 + 개체명 태그를 레이블한 문장' 형태로 읽어들입니다. `NERCorpus`는 `NERDataset`이 요구하면 원본 문장과 레이블한 문장을 `NERDataset`에 제공합니다.

In [9]:
#코드 6-9 학습 데이터셋 구축
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/ner/train.txt
INFO:ratsnlp:processing NER tag dictionary...
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:tokens: [CLS] 이어 옆 ##으로 움직 ##여 김일성 ##의 오른 ##쪽에서 한 차례 ##씩 두 번 상 ##체를 굽 ##혀 조문 ##했 ##으며 이 ##윽 ##고 안 ##경을 벗고 손 ##수 ##건으로 눈 ##주 ##위를 닦 ##기도 했다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O O B-PER O O O B-NOH I-NOH O B-NOH I-NOH O O O O O O O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

`NERDataset` 클래스는 `NERCorpus`와 코드 6-8에서 선언해 둔 토크나이저를 품고 있습니다.  
`NERDataset`은 `NERCorpus`가 넘겨 준 데이터(원본 문장, 레이블한 문장)를 모델이 학습할 수 있는 형태로 가공합니다. 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블한 문장을 모델이 읽어들일 수 있는 포맷(`NERFeatures`)으로 바꿔 주는 역할을 합니다.

`NERFeatures`라는 자료형에는 네 가지 정보가 있습니다.  
첫 번째는 `input_ids`입니다. 인덱스로 변환된 토큰 시퀀스입니다. 두 번째는 `attention_mask`로 해당 토큰이 패딩 토큰인지(`0`), 아닌지(`1`)를 나타냅니다. `token_type_ids`는 세그먼트, `label_ids`은 정수로 바뀐 레이블 시퀀스입니다. 

`NERFeatures` 각 구성 요소의 자료형은 다음과 같습니다.  

- **input_ids**: List[int]
- **attention_mask**: List[int]
- **token_type_ids**: List[int]
- **label_ids**: List[int]

그러면 말뭉치의 3번 데이터를 살펴봅시다.   
다음과 같습니다. 여기서 RS는 구분자이며, 이해를 돕고자 개체명에 해당하는 부분을 별도 색상으로 표시했습니다.  

> ㅡ효진 역의 김환희(14)가 특히 인상적이었다.$_{RS}$ㅡ**<효진:PER>** 역의 **<김환희:PER>**(**<14:NOH>**)가 특히 인상적이었다.

이 내용에 대응하는 `NERCorpus` 데이터는 다음과 같습니다.  
- **text**: ㅡ효진 역의 김환희(14)가 특히 인상적이었다.  
- **label**: ㅡ<효진:PER>역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다.

`NERDataset`은 우선 원본 문장(`text`)을 토큰화합니다.  
그 결과는 다음과 같습니다. 원본 문장의 토큰 시퀀스 앞뒤에 각각 `[CLS]`와`[SEP]`를 붙이고 코드 6-3의 `max_seq_length`(=64)가 되도록 패딩 토큰을 뒷부분에 추가했습니다.

이후 `NERDataset`은 레이블한 문장(`label`)을 바탕으로 레이블 시퀀스를 만듭니다. 문장 앞뒤에 각각 `[CLS]`와 `[SEP]`를 붙이고 코드 6-3의 `max_seq_length`(=64)가 되도록 패딩 토큰을 추가하는 원칙은 이전과 같습니다.   
레이블한 문장을 보면 개체명은 총 3개가 있는데요, `<효진:PER>`, `<김환희:PER>`, `<14:NOH>`이 바로 그것입니다.